In [1]:
import torch

In [2]:
from d2l import torch as d2l
from torch import nn

In [24]:
def corr2d(X, K):
    # 二维运算 构建一个新的张量填充技术后的值
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

In [19]:
K = torch.arange(8).reshape(2, 4)

In [20]:
X = torch.arange(9).reshape(3, 3)
K = torch.arange(4).reshape(2, 2)

In [21]:
X, K

(tensor([[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]]),
 tensor([[0, 1],
         [2, 3]]))

In [25]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [26]:
class Conv2D(nn.Module): # h x w
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [27]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [30]:
K = torch.tensor([[-1.0, 1.0]])

In [31]:
Y = corr2d(X, K)
Y

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [32]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [33]:
X.t()

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [34]:
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)
# 这个⼆维卷积层使⽤四维输⼊和输出格式（批量⼤⼩、通道、⾼度、宽度），
# 其中批量⼤⼩和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
X, Y

(tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.]]]]),
 tensor([[[[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
           [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
           [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
           [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
           [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
           [ 0., -1.,  0.,  0.,  0.,  1.,  0.]]]]))

In [40]:
lr = 3e-2 # 学习率
for i in range(30):
    Y_hat = conv2d(X) # 对x进行卷积运算
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 0.000
epoch 4, loss 0.000
epoch 6, loss 0.000
epoch 8, loss 0.000
epoch 10, loss 0.000
epoch 12, loss 0.000
epoch 14, loss 0.000
epoch 16, loss 0.000
epoch 18, loss 0.000
epoch 20, loss 0.000
epoch 22, loss 0.000
epoch 24, loss 0.000
epoch 26, loss 0.000
epoch 28, loss 0.000
epoch 30, loss 0.000


In [39]:
conv2d.weight.data.reshape((1, 2))

tensor([[-1.0005,  0.9995]])

In [41]:
conv2d.weight.data.reshape((1, 2))

tensor([[-1.0000,  1.0000]])